# Datorseenede projekt 
#### Projektet i kursen datorseende

### Installationer
Installerar paket som behövs för exekveringen av koden.

In [2]:
%pip install split-folders
%pip install --upgrade ultralytics
%pip uninstall -y torch torchvision torchaudio
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Found existing installation: torch 2.6.0+cu126
Uninstalling torch-2.6.0+cu126:
  Successfully uninstalled torch-2.6.0+cu126
Found existing installation: torchvision 0.21.0
Uninstalling torchvision-0.21.0:
  Successfully uninstalled torchvision-0.21.0
Found existing installation: torchaudio 2.6.0+cu126
Uninstalling torchaudio-2.6.0+cu126:
  Successfully uninstalled torchaudio-2.6.0+cu126
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu126
  Using cached https://download.pytorch.org/whl/cu126/torch-2.6.0%2Bcu126-cp313-cp313-win_amd64.whl.metadata (28 kB)
  Using cached https://download.pytorch.org/whl/cu126/torchaudio-2.6.0%2Bcu126-cp313-cp313-win_amd64.whl.metadata (6.8 kB)
Using cached https://download.pytorch.org/whl/cu126/torch-2.6.0%2Bcu126-cp313-cp313-win_amd64.whl (2496.1 MB)
   

In [3]:
import torch
import torchvision
print(torch.__version__)  # Ska vara 2.6.0+cu126
print(torchvision.__version__)  # Ska ha +cu126, inte +cpu
print(torch.cuda.is_available())  # Ska vara True


2.6.0+cu126
0.21.0+cu126
True


### Spjälka data 
Spjälker data för att kunna användas i yolo modellen. Data spjälks till test, train och val.

In [5]:
import splitfolders

input_folder = "./input-data"
output_folder = "./dataset"

splitfolders.ratio(input_folder,output = output_folder, seed=42, ratio=(0.8,0.1,0.1),group_prefix=None)

Copying files: 2762 files [00:03, 827.66 files/s]


## Utför träningen 
Träna en yolov11 model med datan som vi har, med passande parametrar för egen dator att köra.

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolo11n.pt")


results = model.train(
    data="config.yaml",        # Din träningsdata
    project="./output",         # Utmatningsmapp
    device='cuda:0',            # Träna på GPU (ändra om du vill använda CPU)
    epochs=10,                 # Antal epoker
    batch=12,                   # Låt YOLO välja den bästa batch-storleken baserat på GPU-minne
    imgsz=480,                  # Bildstorlek, 640x640 bör vara bra för din GPU
    amp=True,                   # Aktivera Automatic Mixed Precision för snabbare träning och mindre minne              # Spara modellen var 5:e epok
    optimizer='adam',           # Använd Adam optimizer för bättre konvergens
    workers=12,                 # Antal arbetare för dataladdning, öka om du har fler CPU-kärnor       
    name="test",          # Namn på träningssessionen
    patience=20,                # Stoppar om ingen förbättring efter 10 epochs
    exist_ok=True,              # Om du vill skriva över tidigare resultat
)
# Evaluate the model's performance on the validation set
results = model.val()

# Export the model to ONNX format
success = model.export(format="onnx")

Ultralytics 8.3.89  Python-3.13.1 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1650 Ti, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=config.yaml, epochs=10, time=None, patience=20, batch=12, imgsz=480, save=True, save_period=-1, cache=False, device=cuda:0, workers=12, project=./output, name=test, exist_ok=True, pretrained=True, optimizer=adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_b

100%|██████████| 755k/755k [00:01<00:00, 768kB/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     


  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256,

train: Scanning D:\Programming\School\cv-25\cv-25-project\dataset\train\labels... 1104 images, 0 backgrounds, 1 corrupt: 100%|██████████| 1104/1104 [00:03<00:00, 339.48it/s]

...


10 epochs completed in 0.088 hours.
Optimizer stripped from output\test\weights\last.pt, 5.4MB
Optimizer stripped from output\test\weights\best.pt, 5.4MB

Validating output\test\weights\best.pt...
Ultralytics 8.3.89  Python-3.13.1 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1650 Ti, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  4.52it/s]


                   all        138        205      0.422      0.705      0.517      0.298
                 skier         84        121      0.496      0.851      0.621      0.368
           snowboarder         72         84      0.347       0.56      0.412      0.229
Speed: 0.5ms preprocess, 3.0ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to output\test
Ultralytics 8.3.89  Python-3.13.1 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1650 Ti, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning D:\Programming\School\cv-25\cv-25-project\dataset\val\labels.cache... 138 images, 1 backgrounds, 0 corrupt: 100%|██████████| 138/138 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:15<00:00,  1.33s/it]


                   all        138        205      0.416      0.761      0.517      0.297
                 skier         84        121      0.474      0.868      0.625       0.37
           snowboarder         72         84      0.358      0.655       0.41      0.224
Speed: 0.6ms preprocess, 9.2ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to output\test
Ultralytics 8.3.89  Python-3.13.1 torch-2.6.0+cu126 CPU (AMD Ryzen 5 4600H with Radeon Graphics)

PyTorch: starting from 'output\test\weights\best.pt' with input shape (1, 3, 480, 480) BCHW and output shape(s) (1, 6, 4725) (5.2 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Retry 1/2 failed: Command 'pip install --no-cache-dir "onnx>=1.12.0" "onnxslim" "onnxruntime-gpu" ' returned non-zero exit status 1.
